In [1]:
import socket
import numpy as np
import pickle as pkl
import sys
import time
# Package imports
from Server import Server
from Client import Client

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
import tensorflow as tf
# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

manually generated dataset. the client ,server parameters should be change based on the dataset.

In [2]:
num_features = 100
X, y = sklearn.datasets.make_blobs(n_samples=1000, n_features=num_features, centers=10)

In [3]:
X1 = X[:,:int(num_features/2)]
y1 = y[:int(num_features/2)]
X2 = X[:,int(num_features/2):]
y2 = y[int(num_features/2):]

In [4]:
num_examples = len(X)
reg_lambda=0.01
iterations = 20

client1 = Client(X1,y,50)
client2 = Client(X2,y,50)

# SERVER_HOST = "192.168.1.161"
# SERVER_PORT = 5001

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((socket.gethostname(), 1234))

data_size = 0
for i in range(iterations):
    
    random_key = np.random.randint(iterations)
    client1.forward()
    encrypt_z1 = client1.send_msg()
    encrypt_z1 += random_key
    client2.forward()
    encrypt_z2 = client2.send_msg()
    encrypt_z2 -= random_key
    serialized_data1 = pkl.dumps([encrypt_z1,y])
    mse_len1 = pkl.dumps(sys.getsizeof(serialized_data1))
    serialized_data2 = pkl.dumps([encrypt_z2,y])
    mse_len2 = pkl.dumps(sys.getsizeof(serialized_data2))
    s.send(mse_len1)
    s.recv(1024)
    s.send(serialized_data1)
    s.recv(1024)
    s.send(mse_len2)
    s.recv(1024)
    s.send(serialized_data2)
    s.recv(1024)
    
    data = b""
    mse_len = s.recv(1024)
    mse_len = pkl.loads(mse_len,encoding="bytes")
    s.send(bytes("client received you message.","utf-8"))
    temp = s.recv(mse_len+1)
    data += temp
    unserialized_data = pkl.loads(data,encoding="bytes")

    client1.backprop(unserialized_data[:,:int(unserialized_data.shape[1]/2)])
    client2.backprop(unserialized_data[:,int(unserialized_data.shape[1]/2):])
    
s.close()